# Object detection with Python and tensorflow

For details (including installation instructions) see: https://www.mygreatlearning.com/blog/object-detection-using-tensorflow

<img src="https://blog.jetbrains.com/wp-content/uploads/2022/01/dog.jpg" alt="" width="500">

## Libraries and settings

In [4]:
# Libraries
import os
import cv2
import numpy as np
import pathlib
import tensorflow as tf
from PIL import Image
from IPython.display import display
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

# Set path
while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')

# Supress warnings
import warnings
warnings.filterwarnings("ignore")

# Get current working directory
print(os.getcwd())

ImportError: cannot import name 'string_int_label_map_pb2' from 'object_detection.protos' (c:\Users\gell\AppData\Local\anaconda3\envs\mlenv\Lib\site-packages\object_detection\protos\__init__.py)

## Functions

In [5]:
def load_model(model_name):
  base_url = 'http://download.tensorflow.org/models/object_detection/'
  model_file = model_name + '.tar.gz'
  model_dir = tf.keras.utils.get_file(
    fname=model_name, 
    origin=base_url + model_file,
    untar=True)
 
  model_dir = pathlib.Path(model_dir)/"saved_model"
  model = tf.saved_model.load(str(model_dir))
  
  return model

def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]
 
  # Run inference
  model_fn = model.signatures['serving_default']
  output_dict = model_fn(input_tensor)
 
  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections
 
  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)

  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
     
  return output_dict

def show_inference_image(model, image_path):
  # The array based representation of the image will be used later 
  # in order to prepare the result image with boxes and labels on it.
  image_np = np.array(Image.open(image_path))

  # Actual detection
  output_dict = run_inference_for_single_image(model, image_np)

  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=4)

  display(Image.fromarray(image_np))

def show_inference_webcam_feed(model, frame):
  # Take frame from webcam feed and convert to array
  image_np = np.array(frame)
  
  # Actual detection
  output_dict = run_inference_for_single_image(model, image_np)
  
  # Visualization of the results of a detection
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=4)
 
  return(image_np)
 

## Path to labels and pre-trained object detection model

In [3]:
# Path to labels
PATH_TO_LABELS = 'models/research/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

# Pre-trained object detection model
model_name = 'ssd_inception_v2_coco_2017_11_17'
detection_model = load_model(model_name)

NameError: name 'label_map_util' is not defined

## Detecting objects on images

In [ ]:
# Path to images
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('models/research/object_detection/test_images')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))

# Show object classification
for image_path in TEST_IMAGE_PATHS:
    print(image_path)
    show_inference_image(detection_model, image_path)

## Detecting objects in real time using a webcam feed

In [ ]:
# Import pre-trained model
model_name = 'ssd_inception_v2_coco_2017_11_17'
detection_model = load_model(model_name)

# Open webcam and start detecting objects
video_capture = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    re,frame = video_capture.read()
    Imagenp=show_inference_webcam_feed(detection_model, frame)
    cv2.imshow('object detection', cv2.resize(Imagenp, (800,600)))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()

### Jupyter notebook --footer info-- (please always provide this at the end of each submitted notebook)

In [ ]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')